In [13]:
import pandas as pd
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import Model
from tensorflow.keras.applications.resnet50 import ResNet50
import keras.utils as image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.layers import GlobalMaxPooling2D
from sklearn.metrics.pairwise import pairwise_distances

# 데이터셋 경로를 실제 경로로 변경해주세요.
DATASET_PATH = r'C:/Users/tiger/OneDrive/바탕 화면/deep_daiv/추천시스템/challenge/옷'
NUM_ROWS = 1000

# 이미지 데이터 로드
categories = []
filenames = []
images = []

df = pd.DataFrame(columns=['filename', 'image'])
for folder in os.listdir(DATASET_PATH):
    folder_path = os.path.join(DATASET_PATH, folder)
    for file in os.listdir(folder_path):
        filenames.append(file)
        images.append(os.path.join(folder_path, file))

df['filename'] = filenames
df['image'] = images
# 파일 경로에서 모든 역슬래시(\)를 슬래시(/)로 변환
df['image'] = df['image'].apply(lambda x: x.replace('\\', '/'))

df = df.iloc[np.random.permutation(df.index)].reset_index(drop=True)
df = df[:NUM_ROWS]

# 이미지 로딩 및 출력 함수
def load_image(img_path, resized_fac=0.1):
    img = cv2.imread(img_path)
    if img is None:
        print(f"Could not read the image from path: {img_path}")
        return None
    w, h, _ = img.shape
    resized = cv2.resize(img, (int(h * resized_fac), int(w * resized_fac)), interpolation=cv2.INTER_AREA)
    return resized

image_to_show = load_image(df.iloc[idx_ref].image)
if image_to_show is not None:
    plt.figure(figsize=(2, 2))
    plt.imshow(cv2.cvtColor(image_to_show, cv2.COLOR_BGR2RGB))
else:
    print("Image could not be loaded and displayed.")


def plot_figures(figures, nrows=1, ncols=1, figsize=(5, 3)):
    fig, axeslist = plt.subplots(ncols=ncols, nrows=nrows, figsize=figsize)
    for ind, title in enumerate(figures):
        axeslist.ravel()[ind].imshow(cv2.cvtColor(figures[title], cv2.COLOR_BGR2RGB))
        axeslist.ravel()[ind].set_title(title)
        axeslist.ravel()[ind].set_axis_off()
    plt.tight_layout()

# ResNet50 모델 구성
img_width, img_height, img_channel = 224, 224, 3
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, img_channel))
base_model.trainable = False

model = tf.keras.Sequential([
    base_model,
    GlobalMaxPooling2D()
])

# 이미지 임베딩 함수
def get_embedding(model, img_path):
    img = image.load_img(img_path, target_size=(img_width, img_height))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return model.predict(x).reshape(-1)

# 전체 데이터 임베딩
map_embeddings = df['image'].apply(lambda img: get_embedding(model, img))
df_embs = map_embeddings.apply(pd.Series)

# 유사도 계산
cosine_sim = 1 - pairwise_distances(df_embs, metric='cosine')

# 유사한 이미지 추천 함수
indices = pd.Series(range(len(df)), index=df.index)

def get_recommender(idx, df, top_n=5):
    sim_idx = indices[idx]
    sim_scores = list(enumerate(cosine_sim[sim_idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n + 1]
    idx_rec = [i[0] for i in sim_scores]
    idx_sim = [i[1] for i in sim_scores]

    return indices.iloc[idx_rec].index, idx_sim

# 예시로 유사한 데이터 출력
idx_ref = 1
idx_rec, idx_sim = get_recommender(idx_ref, df, top_n=6)

plt.figure(figsize=(2, 2))
plt.imshow(cv2.cvtColor(load_image(df.iloc[idx_ref].image), cv2.COLOR_BGR2RGB))

figures = {'img-' + str(i): load_image(row.image) for i, row in df.loc[idx_rec].iterrows()}
plot_figures(figures, 2, 3)


Could not read the image from path: C:/Users/tiger/OneDrive/바탕 화면/deep_daiv/추천시스템/challenge/옷/상의/987.jpg
Image could not be loaded and displayed.
1/1 [==============================] - 1s 577ms/step


1/1 [==============================] - 0s 434ms/step


1/1 [==============================] - 1s 517ms/step


1/1 [==============================] - 1s 529ms/step


1/1 [==============================] - 1s 524ms/step


1/1 [==============================] - 0s 493ms/step


1/1 [==============================] - 1s 527ms/step
Could not read the image from path: C:/Users/tiger/OneDrive/바탕 화면/deep_daiv/추천시스템/challenge/옷/상의/987.jpg


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


<Figure size 200x200 with 0 Axes>